In [6]:
"""
Implementation of probabalistic (Beta-Geometric/NBD) generative modeling, coupled with Bayesian
tecnhiques for parameter estimation and model tuning. Using these means the end goal of achieving
meaningful, accurate predictions for customer-level statistics in a business with a non-contractual,
continuous-time revenue model has finally been realized. Predictions for total LTV, future
transaction flows, time-until-next-purchase, and time-until-dropout may all be estimated 
accurately and at customer-level granularity.
"""

__author__ = 'dm'


from math import log, exp
import numpy as np
from scipy.optimize import minimize
from scipy.special import gammaln


In [7]:
def log_likelihood_individual(r, alpha, a, b, x, tx, t):
    
    """Log-likelihood function for a randomly chosen individual
    with purchase history H = (x, t, T), where x is the number
    of transactions in the time interval (0, T], and t (0 < t <= T)
    is the time of the most recent transaction.
    """
    
    ln_a1 = gammaln(r + x) - gammaln(r) + r * log(alpha)
    ln_a2 = gammaln(a + b) + gammaln(b + x) - gammaln(b) - gammaln(a + b + x)
    ln_a3 = -(r + x) * log(alpha + t)
    a4 = 0
    if x > 0:
        a4 = exp(log(a) - log(b + x - 1) - (r + x) * log(alpha + tx))
    return ln_a1 + ln_a2 + log(exp(ln_a3) + a4)


def log_likelihood(r, alpha, a, b, customers):
    """Sum of the individual log likelihoods"""
    # can't put constraints on n-m minimizer so fake them here
    if r <= 0 or alpha <= 0 or a <= 0 or b <= 0:
        return -np.inf
    return sum([log_likelihood_individual(r, alpha, a, b, x, tx, t) for x, tx, t in customers])


def maximize(customers):
    negative_ll = lambda params: -log_likelihood(*params, customers=customers)
    params0 = np.array([1., 1., 1., 1.])
    res = minimize(negative_ll, params0, method='nelder-mead', options={'xtol': 1e-8})
    return res


def fit(customers):
    res = maximize(customers)
    if res.status != 0:
        raise Exception(res.message)
    return res.x


def cdnow_customer_data(fname):
    data = []
    with open(fname) as f:
        f.readline()
        for line in f:
            data.append(map(float, line.strip().split(',')[1:4]))
    return data


def main():
    data = cdnow_customer_data('cdnow_customers.csv')
    r, alpha, a, b = fit(data)
    # fit according to the note
    print r, alpha, a, b
    print np.allclose([r, alpha, a, b], [.243, 4.414, .793, 2.426], 1e-2)


if __name__ == '__main__':
    main()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-7-e1f81fbee407>, line 53)